# Bağlı, ilişkisiz ve statik metotlar

Python 3'te, bir sınıf içinde bir metot bildirdiğinizde, bir def anahtar sözcüğünü kullanırsınız, böylece bir fonksiyon nesnesi oluşturursunuz. Bu normal bir fonksiyondur ve çevresindeki sınıf, onun ad alanı (namespace) olarak çalışır. Aşağıdaki örnekte, A sınıfı içinde f yöntemini bildiriyoruz ve bu bir A işlevi haline geliyor:

In [37]:
class A(object):
    def f(self, x):
        return 2 * x
A.f

<function __main__.A.f(self, x)>

Şimdi inspect modülü ile fonksiyon ve metot ayrıma bakalım

In [38]:
import inspect
print(inspect.isfunction(A.f))

print(inspect.ismethod(A.f))

True
False


Python fonksiyonunun / metodunun, ilk argüman olarak A sınıfının bir örneğini iletmeniz koşuluyla, doğrudan çağrılabilir.

In [39]:
A.f(1, 7)

14

In [40]:
a = A()
A.f(a, 20)

40

Şimdi, a'nın A sınıfının bir örneği olduğunu varsayalım, o zaman a.f nedir? Pekala, sezgisel olarak bu, A sınıfının f metodu ile aynı olmalıdır, ancak bir şekilde bunun a nesnesine uygulandığını "bilmesi" gerekir - Python'da buna a'ya bağlı metot denir. Nitelikli ayrıntılar şu şekildedir: a.f yazmak, a'nın sihirli __ getattribute __ yöntemini çağırır, bu ilk önce a'nın f adında bir özniteliğe sahip olup olmadığını kontrol eder, ardından A sınıfının böyle bir ada sahip bir yöntem içerip içermediğini kontrol eder  ve m.__ func__ cinsinden orijinal A.f'ye ve m.__ self__ içindeki nesneye referansa sahip yeni bir m tipi yöntem nesnesi oluşturur. Bu nesne bir fonksiyon olarak çağrıldığında, basitçe şunu yapar: m (...) => m .__ func __ (m .__ self__, ...). Bu nedenle, bu nesneye bağımlı yöntem denir çünkü çağrıldığında, ilk argüman olarak bağlı olduğu nesneyi sağlamayı bilir.

In [41]:
a = A()
a.f

<bound method A.f of <__main__.A object at 0x0000022BFF5AE3A0>>

In [42]:
a.f(2)

4

In [43]:
a.f is a.f #bağlı metot nesnesi a.f, * her çağırıldığında * yeniden oluşturulur:

False

In [44]:
a.f = a.f

In [45]:
a.f is a.f

True

Son olarak, Python'un sınıf metotları ve statik metotları vardır - özel tür yöntemler. Sınıf metotları, bir nesne üzerinde çağrıldıklarında nesne yerine nesnenin sınıfına bağlanmaları dışında normal metotlarla aynı şekilde çalışır. Böylece m.__self__ = type(a) diyebiliriz. Böyle bir bağımlı yöntemi çağırdığınızda, ilk argüman olarak a sınıfını iletir. Statik yöntemler daha da basittir: hiçbir şeyi bağlamazlar ve herhangi bir dönüşüm olmadan temeldeki fonksiyonu döndürürler.

In [55]:
class A(object):
    def foo(self, x):
        print(f"executing foo({self}, {x})")

    @classmethod
    def class_foo(cls, x):
        print(f"executing class_foo({cls}, {x})")

    @staticmethod
    def static_foo(x):
        print(f"executing static_foo({x})")

a = A()

Aşağıda, bir nesne örneğinin bir metodu çağırmasının olağan yolu verilmiştir. Nesne örneği, ailk argüman olarak örtük olarak iletilir.

In [56]:
a.foo(1)

executing foo(<__main__.A object at 0x0000022BFF664D00>, 1)


classmethodlar ile , nesne örneğinin sınıfı, self yerine ilk argüman olarak örtük olarak iletilir.

Ayrıca sınıfı kullanarak class_foo'yu da çağırabilirsiniz. Aslında, bir şeyi classmethod olarak tanımlarsanız, bunun nedeni muhtemelen onu bir sınıf örneğinden ziyade sınıftan çağırmak istemenizdir. A.foo (1) bir TypeError oluşturabilirdi, ancak A.class_foo (1) gayet iyi çalışıyor:

In [58]:
A.class_foo(1)

executing class_foo(<class '__main__.A'>, 1)


Statik yöntemlerle, ne self (nesne örneği) ne de cls (sınıf) ilk bağımsız değişken olarak örtük olarak iletilir. Onları bir örnekten veya sınıftan çağırabilmeniz dışında düz fonksiyonlar gibi davranırlar:

In [59]:
a.static_foo(1)

executing static_foo(1)


In [60]:
a.static_foo("hi")

executing static_foo(hi)


Statik metotlar, sınıfla mantıksal bağlantısı olan fonksiyonları gruplamak için kullanılır.

foo sadece bir fonksiyondur, ancak a.foo'yu çağırdığınızda, sadece fonksiyonu elde etmezsiniz, fonksiyonun "kısmen uygulanmış" bir sürümünü, nesne örneğinin işlevin ilk argümanı olarak bağlı olduğu bir sürümünü elde edersiniz.




foo sadece bir fonksiyondur, ancak aradığınızda, yalnızca işlevi a.fooalmıyorsunuz, işlevin ailk bağımsız değişkeni olarak nesne örneğine bağlı olarak işlevin "kısmen uygulanmış" bir sürümünü elde edersiniz. foo 2 argüman beklerken a.foo sadece 1 argüman bekler.

a foo ya bağlıdır. BU yüzden "bağlı",(bound) terimi aşağıdaki gibidir:

In [61]:
print(a.foo)

<bound method A.foo of <__main__.A object at 0x0000022BFF664D00>>


a.class_foo ile, a, class_foo'ya bağlanmaz, bundan ziyade, A class_foo'ya bağlanır.

In [63]:
print(a.class_foo)

<bound method A.class_foo of <class '__main__.A'>>


staticmethod ile, aslında adının metot olmasına rağmen, a.static_foo bağlı argümanı olmayan bir fonksiyon olarak döner. a.static_foo sadece bir argüman bekler.

In [64]:
print(a.static_foo)

<function A.static_foo at 0x0000022BFF66D040>


Ve tabii ki bunun yerine A sınıfı ile static_foo arandığında da aynı şey olur.

In [65]:
print(A.static_foo)

<function A.static_foo at 0x0000022BFF66D040>


## Örnek

In [67]:
class Person(object):
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age
        self.full_name = first_name + " " + last_name
    def greet(self):
        print("Hello, my name is " + self.full_name + ".")

Ad ve soyad yerine ayrı ayrı tam ad belirterek bu sınıfın örneklerini oluşturmanın bir yolunun olması kullanışlı olabilir. Bunu yapmanın bir yolu, soyadının isteğe bağlı bir parametre olması ve eğer verilmemişse, tam adı şurada geçtiğimizi varsaymaktır:

In [68]:
class Person(object):
    def __init__(self, first_name, age, last_name=None):
        if last_name is None:
            self.first_name, self.last_name = first_name.split(" ", 2)
        else:
            self.first_name = first_name
            self.last_name = last_name
            self.full_name = self.first_name + " " + self.last_name
            self.age = age
    def greet(self):
        print("Hello, my name is " + self.full_name + ".")

Bununla birlikte, bu kod parçasıyla ilgili iki ana sorun vardır:

1. First_name için tam bir ad girebildiğiniz için, ad ve soyadı parametreleri artık yanıltıcıdır. Ayrıca, bu tür esnekliğe sahip daha fazla durum ve/veya daha fazla parametre varsa, if/elif/else dallanması can sıkıcı hale gelebilir.

2. O kadar önemli değil, ama yine de belirtmeye değer: ya last_name None ise, ancak first_name boşluklar aracılığıyla iki veya daha fazla şeye ayrılmıyorsa? Yine başka bir girdi doğrulama ve / veya istisna işleme katmanımız var ...

classmethod girin. Tek bir başlatıcıya sahip olmak yerine, tam_adı denilen ayrı bir başlatıcı oluşturacağız ve onu (yerleşik) sınıf yöntem dekoratörü ile süsleyeceğiz.

In [71]:
class Person(object):
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age
        self.full_name = first_name + " " + last_name
        
    @classmethod
    def from_full_name(cls, name, age):
        if " " not in name:
            raise ValueError
        first_name, last_name = name.split(" ", 2)
        return cls(first_name, last_name, age)
        
    def greet(self):
        print("Hello, my name is " + self.full_name + ".")

From_full_name için ilk argüman olarak self yerine cls'ye dikkat edin. Sınıf yöntemleri, belirli bir sınıfın bir örneğine değil, genel sınıfa uygulanır (bu, genellikle self'in ifade ettiği şeydir). Dolayısıyla, cls bizim Person sınıfımızsa, from_full_name sınıf metodundan döndürülen değer, Person sınıfının bir örneğini oluşturmak için Person'ın __init__'ini kullanan Person(ad_name, last_name, age) olur. Özellikle, Çalışanın bir alt sınıfını yapacak olsaydık, from_full_name Çalışan sınıfında da çalışırdı.
Bunun beklendiği gibi çalıştığını göstermek için, __init__ içinde dallanma olmadan birden fazla şekilde Person örnekleri oluşturalım:

In [72]:
bob = Person("Bob", "Bobberson", 42)

In [73]:
alice = Person.from_full_name("Alice Henderson", 31)

In [74]:
bob.greet()

Hello, my name is Bob Bobberson.


In [75]:
alice.greet()

Hello, my name is Alice Henderson.
